##**YOLO**

In [ ]:
!pip install ultralytics
!pip install onnxruntime-gpu
!pip install onnx
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.7/871.7 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.2/226.2 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 84.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 81.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO("yolov8l.pt")
results = model("df.jpg")

for result in results:
  result.save(filename="result.jpg")



image 1/1 /content/df.jpg: 384x640 1 cup, 1 potted plant, 1 tv, 2 mouses, 1 keyboard, 2001.1ms
Speed: 7.1ms preprocess, 2001.1ms inference, 25.6ms postprocess per image at shape (1, 3, 384, 640)


In [ ]:

model.export(format="onnx")


Ultralytics YOLOv8.2.89 🚀 Python-3.10.12 torch-2.4.0+cu121 CPU (Intel Xeon 2.20GHz)

PyTorch: starting from 'yolov8l.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (83.7 MB)

ONNX: starting export with onnx 1.16.2 opset 19...
ONNX: export success ✅ 6.3s, saved as 'yolov8l.onnx' (166.8 MB)

Export complete (14.9s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8l.onnx imgsz=640  
Validate:        yolo val task=detect model=yolov8l.onnx imgsz=640 data=coco.yaml  
Visualize:       https://netron.app


'yolov8l.onnx'

In [ ]:
onnx_model = YOLO("yolov8l.onnx")
result = onnx_model("df.jpg")
for result in results:
  result.save(filename="result1.jpg")

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
Loading yolov8l.onnx for ONNX Runtime inference...

image 1/1 /content/df.jpg: 640x640 1 cup, 1 potted plant, 1 tv, 2 mouses, 1 keyboard, 2937.1ms
Speed: 3.2ms preprocess, 2937.1ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


##Scikit_Learn

In [ ]:
!pip install skl2onnx


In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
X, y = iris.data, iris.target
X = X.astype(np.float32)
X_train, X_test,y_train, y_test = train_test_split(X,y)

model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [ ]:
import skl2onnx
onnx_model=skl2onnx.to_onnx(model,X[0])
with open("scikit_ONNX","wb")as f:
  f.write(onnx_model.SerializeToString())

##PyTorch

In [ ]:
!pip install onnx
!pip install onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.2/671.2 kB 8.3 MB/s eta 0:00:00


In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MyModel(nn.Module):

    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
!pip install --upgrade onnxscript

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 671.2/671.2 kB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 87.8 MB/s eta 0:00:00


In [9]:
torch_model = MyModel()
torch_input = torch.randn(1, 1, 32, 32)
onnx_program = torch.onnx.dynamo_export(torch_model, torch_input)

/usr/local/lib/python3.10/dist-packages/torch/onnx/_internal/exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


In [10]:
onnx_program.save("my_image_classifier.onnx")


In [11]:
import onnx
onnx_model = onnx.load("my_image_classifier.onnx")
onnx.checker.check_model(onnx_model)

In [15]:
!pip install onnxruntime

##Tensorflow

In [17]:
!pip install tf2onnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.8/455.8 kB 20.2 MB/s eta 0:00:00


In [18]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50


model = ResNet50(weights="imagenet", include_top=True)


102967424/102967424 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [19]:
import tf2onnx

input_data = tf.TensorSpec(model.inputs[0].shape,
                  dtype=model.inputs[0].dtype,
                  name=model.inputs[0].name)

onnx_model, _ = tf2onnx.convert.from_keras(
    model,
    [input_data])

In [20]:
onnx.save(onnx_model, "resnet50_model.onnx")